In [907]:

#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [908]:
#Carga desde un archivo .csv sin indice
df = pd.read_csv('Base_Mexico_LIMPIA.csv')
df.head(5)

,Unnamed: 0,id,scrape_id,host_id,latitude,longitude,listing_url,last_scraped,source,name,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,reviews_per_month
0,0,898633,20240627045222,4796027,19.42063,-99.16586,https://www.airbnb.com/rooms/898633,2024-06-28,city scrape,City SkyLine View from 1 Bedroom Apt in CDMX,...,4.95,4.93,4.95,4.93,4.96,4.93,4.91,2.0,1.0,0.41
1,1,35797,20240627045222,153786,19.38283,-99.27178,https://www.airbnb.com/rooms/35797,2024-06-27,city scrape,Villa Dante,...,4.80,4.90,4.80,4.90,4.90,4.90,4.80,1.0,0.0,1.50
2,2,44616,20240627045222,196253,19.41162,-99.17794,https://www.airbnb.com/rooms/44616,2024-06-28,city scrape,CONDESA HAUS B&B,...,4.58,4.56,4.70,4.87,4.78,4.98,4.48,4.0,2.0,0.42
3,3,913084,20240627045222,4647293,19.43956,-99.17263,https://www.airbnb.com/rooms/913084,2024-06-28,city scrape,15 blocks from Paseo de la Reforma,...,4.75,4.83,4.72,4.84,4.93,4.90,4.75,1.0,0.0,1.19
4,4,67703,20240627045222,334451,19.41152,-99.16857,https://www.airbnb.com/rooms/67703,2024-06-28,city scrape,"2 bedroom apt. deco bldg, Condesa",...,4.90,4.81,4.75,4.94,4.92,4.98,4.91,2.0,0.0,0.32


In [909]:
#Sustituir valores nulos por un string en  concreto 
df["host_response_rate"] =df["host_response_rate"].replace('No captured',0) 
# Elimina cualquier símbolo no numérico (por ejemplo, '%') y convierte a tipo float
df['host_response_rate'] = df['host_response_rate'].str.replace('%', '').astype(float)

#Sustituir valores nulos por un string en  concreto 
df["host_acceptance_rate"] =df["host_acceptance_rate"].replace('No captured',0) 
# Elimina cualquier símbolo no numérico (por ejemplo, '%') y convierte a tipo float
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.replace('%', '').astype(float)

#Realizamos una copia del dataframe
data=df.copy()

#Sustituir valores nulos con promedio o media
df["host_response_rate"]=df["host_response_rate"].fillna(round(data["host_response_rate"].mean(),1))
df["host_acceptance_rate"]=df["host_acceptance_rate"].fillna(round(data["host_acceptance_rate"].mean(),1))

In [910]:
#Rellenamos nulos
df =df.fillna(method="bfill")
df =df.fillna(method="ffill")

C:\Users\sarah\AppData\Local\Temp\ipykernel_29972\1655763348.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="bfill")
C:\Users\sarah\AppData\Local\Temp\ipykernel_29972\1655763348.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="ffill")


In [911]:
nulos=df.isnull().sum().sum()
nulos

0

In [912]:
#ajustar max de filas
pd.options.display.max_rows=10
cuantitativas=df.iloc[ : , 40:71]
cuantitativas

,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,...,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms
0,3.0,6.0,2.0,1.0,1.0,2.0,1652.0,3.0,48,3.0,...,0.0,4.95,4.93,4.95,4.93,4.96,4.93,4.91,2.0,1.0
1,1.0,1.0,2.0,1.0,1.0,1.0,1057.7,1.0,7,1.0,...,0.0,4.80,4.90,4.80,4.90,4.90,4.90,4.80,1.0,0.0
2,17.0,17.0,2.9,1.4,1.4,1.5,1057.7,1.0,365,1.0,...,0.0,4.58,4.56,4.70,4.87,4.78,4.98,4.48,4.0,2.0
3,1.0,4.0,4.0,1.0,2.0,3.0,814.0,2.0,1125,2.0,...,0.0,4.75,4.83,4.72,4.84,4.93,4.90,4.75,1.0,0.0
4,3.0,4.0,4.0,1.0,2.0,2.0,1923.0,2.0,30,2.0,...,0.0,4.90,4.81,4.75,4.94,4.92,4.98,4.91,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26531,13.0,15.0,2.9,1.4,1.4,1.5,1057.7,2.0,365,2.0,...,0.0,4.80,4.90,4.80,4.90,4.90,4.90,4.80,8.0,0.0
26532,33.0,7.1,3.0,1.0,1.0,2.0,1478.0,1.0,365,1.0,...,0.0,4.80,4.90,4.80,4.90,4.90,4.90,4.80,11.0,0.0
26533,12.0,15.0,3.0,1.0,1.0,1.0,936.0,2.0,365,2.0,...,0.0,4.80,4.90,4.80,4.90,4.90,4.90,4.80,1.0,0.0
26534,3.0,3.0,4.0,1.5,1.0,2.0,768.0,1.0,365,1.0,...,0.0,4.80,4.90,4.80,4.90,4.90,4.90,4.80,3.0,0.0


In [913]:
variables=df.iloc[ : , [19,20,41,42,71,46]]
variables

,host_response_rate,host_acceptance_rate,host_total_listings_count,accommodates,reviews_per_month,price
0,100.0,88.0,6.0,2.0,0.41,1652.0
1,0.0,89.8,1.0,2.0,1.50,1057.7
2,100.0,92.0,17.0,2.9,0.42,1057.7
3,100.0,96.0,4.0,4.0,1.19,814.0
4,89.0,62.0,4.0,4.0,0.32,1923.0
...,...,...,...,...,...,...
26531,100.0,100.0,15.0,2.9,1.50,1057.7
26532,97.0,91.0,7.1,3.0,1.50,1478.0
26533,100.0,100.0,15.0,3.0,1.50,936.0
26534,100.0,100.0,3.0,4.0,1.50,768.0


PRICE

In [914]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['price'].max()
Min=df['price'].min()
Limites= [Min, Max]
Limites

[105.0, 2980.0]

In [915]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(107.0, 2981.0, 3)
intervalos

array([ 107., 1544., 2981.])

In [916]:
#Creamos las categorías 
categorias= ["Min.Price", "Max.Price"]

In [917]:
#Finalmente creamos las categorías en la columna numérica
df['price']=pd.cut(x= df['price'], bins=intervalos, labels= categorias )
df['price']

0        Max.Price
1        Min.Price
2        Min.Price
3        Min.Price
4        Max.Price
           ...    
26531    Min.Price
26532    Min.Price
26533    Min.Price
26534    Min.Price
26535    Min.Price
Name: price, Length: 26536, dtype: category
Categories (2, object): ['Min.Price' < 'Max.Price']

In [918]:
df['price'].isnull().sum()

1

In [919]:
df=df.dropna(subset=['price'])

In [920]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['bathrooms', 'bedrooms', 'beds']]
Var_Dep= df['price']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

In [921]:
#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [922]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['Min.Price', 'Min.Price', 'Min.Price', ..., 'Min.Price',
       'Min.Price', 'Min.Price'], dtype=object)

In [923]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

Matriz de Confusión
[[  26  795]
 [  48 4438]]


In [924]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.8480794955092681


In [925]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8411531938948559


In [926]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9893000445831476


Reviews_per_month

In [927]:
df['reviews_per_month'].isnull().sum()

0

In [928]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['reviews_per_month'].max()
Min=df['reviews_per_month'].min()
Limites= [Min, Max]
Limites

[0.01, 5.39]

In [929]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(0.02, 5.40, 3)
intervalos

array([0.02, 2.71, 5.4 ])

In [930]:
#Creamos las categorías 
categorias= ["Min.Rewiews", "Max.Rewiews"]

#Finalmente creamos las categorías en la columna numérica
df['reviews_per_month']=pd.cut(x= df['reviews_per_month'], bins=intervalos, labels= categorias )
df['reviews_per_month']

0        Min.Rewiews
1        Min.Rewiews
2        Min.Rewiews
3        Min.Rewiews
4        Min.Rewiews
            ...     
26531    Min.Rewiews
26532    Min.Rewiews
26533    Min.Rewiews
26534    Min.Rewiews
26535    Min.Rewiews
Name: reviews_per_month, Length: 26535, dtype: category
Categories (2, object): ['Min.Rewiews' < 'Max.Rewiews']

In [931]:
df['reviews_per_month'].isnull().sum()

226

In [932]:
df=df.dropna(subset=['reviews_per_month'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26309 entries, 0 to 26535
Data columns (total 72 columns):
 #   Column                                        Non-Null Count  Dtype   
---  ------                                        --------------  -----   
 0   Unnamed: 0                                    26309 non-null  int64   
 1   id                                            26309 non-null  int64   
 2   scrape_id                                     26309 non-null  int64   
 3   host_id                                       26309 non-null  int64   
 4   latitude                                      26309 non-null  float64 
 5   longitude                                     26309 non-null  float64 
 6   listing_url                                   26309 non-null  object  
 7   last_scraped                                  26309 non-null  object  
 8   source                                        26309 non-null  object  
 9   name                                          26309 non

In [933]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['review_scores_cleanliness', 'review_scores_communication', 'review_scores_location', 'review_scores_value']]
Var_Dep= df['reviews_per_month']
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [934]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['Min.Rewiews', 'Min.Rewiews', 'Min.Rewiews', ..., 'Min.Rewiews',
       'Min.Rewiews', 'Min.Rewiews'], dtype=object)

In [935]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

Matriz de Confusión
[[   0  789]
 [   0 4473]]


In [936]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Rewiews")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.8500570125427594


In [937]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8500570125427594


In [938]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Rewiews")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


PRICE

In [939]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['price'].max()
Min=df['price'].min()
Limites= [Min, Max]
Limites

['Min.Price', 'Max.Price']

In [940]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(104.0, 2981.0, 3)
intervalos

array([ 104. , 1542.5, 2981. ])

In [941]:
#Creamos las categorías 
categorias= ["Min.Price", "Max.Price"]

In [942]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['minimum_nights', 'maximum_nights']]
Var_Dep= df['price']
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [943]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['Min.Price', 'Min.Price', 'Min.Price', ..., 'Min.Price',
       'Min.Price', 'Min.Price'], dtype=object)

In [944]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

Matriz de Confusión
[[   0  876]
 [   0 4386]]


In [945]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.8335233751425314


In [946]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8335233751425314


In [947]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


Availability_365

In [948]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['availability_365'].max()
Min=df['availability_365'].min()
Limites= [Min, Max]
Limites

[0, 365]

In [949]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(0, 365, 3)
intervalos

array([  0. , 182.5, 365. ])

In [950]:
#Creamos las categorías 
categorias= ["Inicio_Año", "Fin_Año"]

#Finalmente creamos las categorías en la columna numérica
df['availability_365']=pd.cut(x= df['availability_365'], bins=intervalos, labels= categorias )
df['availability_365']

0        Fin_Año
1        Fin_Año
2        Fin_Año
3        Fin_Año
4        Fin_Año
          ...   
26531    Fin_Año
26532    Fin_Año
26533    Fin_Año
26534    Fin_Año
26535    Fin_Año
Name: availability_365, Length: 26309, dtype: category
Categories (2, object): ['Inicio_Año' < 'Fin_Año']

In [951]:
df=df.dropna(subset=['availability_365'])

In [952]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['minimum_nights', 'maximum_nights']]
Var_Dep= df['availability_365']
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [953]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['Fin_Año', 'Fin_Año', 'Fin_Año', ..., 'Fin_Año', 'Inicio_Año',
       'Fin_Año'], dtype=object)

In [954]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

Matriz de Confusión
[[3036  174]
 [1500  133]]


In [955]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Fin_Año")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.6693121693121693


In [956]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6543464794548833


In [957]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Fin_Año")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9457943925233645


host_total_listings_count

In [959]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['host_total_listings_count'].max()
Min=df['host_total_listings_count'].min()
Limites= [Min, Max]
Limites

[1.0, 39.0]

In [960]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(2.0, 40, 3)
intervalos

array([ 2., 21., 40.])

In [961]:
#Creamos las categorías 
categorias= ["Min.Count", "Max.Count"]

#Finalmente creamos las categorías en la columna numérica
df['host_total_listings_count']=pd.cut(x= df['host_total_listings_count'], bins=intervalos, labels= categorias )
df['host_total_listings_count']

0        Min.Count
1              NaN
2        Min.Count
3        Min.Count
4        Min.Count
           ...    
26531    Min.Count
26532    Min.Count
26533    Min.Count
26534    Min.Count
26535    Min.Count
Name: host_total_listings_count, Length: 24211, dtype: category
Categories (2, object): ['Min.Count' < 'Max.Count']

In [962]:
df=df.dropna(subset=['host_total_listings_count'])

In [964]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['latitude', 'longitude', 'availability_30']]
Var_Dep= df['host_total_listings_count']
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [965]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['Min.Count', 'Min.Count', 'Min.Count', ..., 'Min.Count',
       'Min.Count', 'Min.Count'], dtype=object)

In [966]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

Matriz de Confusión
[[   0  338]
 [   0 3055]]


In [967]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Count")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9003831417624522


In [968]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9003831417624522


In [969]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Count")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


*Variables Categóricas*